In [5]:
import pandas as pd
gss = pd.read_csv('gss_bayes.csv')


In [3]:
import os
print(os.getcwd())



C:\Users\david


In [6]:
gss.head()


,caseid,year,age,sex,polviews,partyid,indus10
0,1,1974,21.0,1,4.0,2.0,4970.0
1,2,1974,41.0,1,5.0,0.0,9160.0
2,5,1974,58.0,2,6.0,1.0,2670.0
3,6,1974,30.0,1,5.0,4.0,6870.0
4,7,1974,48.0,1,5.0,4.0,7860.0


In [4]:
!pip install pyarrow


   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.1 MB 187.9 kB/s eta 0:02:14
   ---------------------------------------- 0.0/25.1 MB 196.9 kB/s eta 0:02:08
   ---------------------------------------- 0.1/25.1 MB 252.2 kB/s eta 0:01:40
   ---------------------------------------- 0.1/25.1 MB 327.7 kB/s eta 0:01:17
   ---------------------------------------- 0.2/25.1 MB 557.9 kB/s eta 0:00:45
    --------------------------------------- 0.6/25.1 MB 1.7 MB/s eta 0:00:15
   -- ------------------------------------- 1.8/25.1 MB 4.4 MB/s eta 0:00:06
   ------ --------------------------------- 3.9/25.1 MB 8.6 MB/s eta 0:00:03
   --------- ------------------------------ 5.8/25.1 MB 11.7 MB/s eta 0:00:02
   ------------ 

In [7]:
banker = (gss['indus10'] == 6870)


In [8]:
banker.head()


0    False
1    False
2    False
3     True
4    False
Name: indus10, dtype: bool

In [9]:
banker_count = banker.sum()
print(f'Number of bankers: {banker_count}')


Number of bankers: 728


In [10]:
banker_fraction = banker.mean()
print(f'Fraction of bankers: {banker_fraction}')


Fraction of bankers: 0.014769730168391155


In [11]:
print(f'Total respondents: {len(gss)}')
print(f'Number of bankers: {banker_count}')
print(f'Fraction of bankers (or probability of being a banker): {banker_fraction * 100:.2f}%')


Total respondents: 49290
Number of bankers: 728
Fraction of bankers (or probability of being a banker): 1.48%


In [12]:
def prob(A):
    """Computes the probability of a proposition, A."""    
    return A.mean()

In [13]:
prob(banker)


0.014769730168391155

In [14]:
female = (gss['sex'] == 2)


In [16]:
prob(female)


0.5378575776019476

In [17]:
liberal = (gss['polviews'] <= 3)


In [18]:
prob(liberal)


0.27374721038750255

In [19]:
democrat = (gss['partyid'] <= 1)


In [20]:
prob(democrat)


0.3662609048488537

In [21]:
prob(banker & democrat)


0.004686548995739501

In [22]:
prob(democrat & banker)


0.004686548995739501

In [23]:
def conditional(proposition, given):
    """Probability of A conditioned on given."""
    return prob(proposition[given])


In [24]:
conditional(democrat, given=liberal)


0.5206403320240125

In [25]:
conditional(liberal, given=democrat)


0.3891320002215698

In [26]:
conditional(female, given=banker)


0.7706043956043956

In [27]:
conditional(banker, given=female)


0.02116102749801969

In [28]:
conditional(female, given=liberal & democrat)


0.576085409252669

In [29]:
conditional(liberal & female, given=banker)


0.17307692307692307

In [30]:
female[banker].mean()

0.7706043956043956

In [31]:
conditional(female, given=banker)

0.7706043956043956

In [32]:
prob(female & banker) / prob(banker)

0.7706043956043956

In [33]:
prob(liberal & democrat)

0.1425238385067965

In [34]:
prob(democrat) * conditional(liberal, democrat)

0.1425238385067965

In [35]:
conditional(liberal, given=banker)

0.2239010989010989

In [36]:
prob(liberal) * conditional(banker, liberal) / prob(banker)

0.2239010989010989

In [37]:
male = (gss['sex'] == 1)

In [38]:
prob(male & banker) + prob(female & banker)

0.014769730168391155

In [39]:
(prob(male) * conditional(banker, given=male) +
prob(female) * conditional(banker, given=female))

0.014769730168391153

In [40]:
B = gss['polviews']
B.value_counts().sort_index()

polviews
1.0     1442
2.0     5808
3.0     6243
4.0    18943
5.0     7940
6.0     7319
7.0     1595
Name: count, dtype: int64

In [41]:
i = 4
prob(B==i) * conditional(banker, B==i)

0.005822682085615744

In [42]:
sum(prob(B==i) * conditional(banker, B==i)
    for i in range(1, 8))

0.014769730168391157

In [43]:
# Probability that a respondent is liberal given that they are a Democrat
conditional(liberal, given=democrat)


0.3891320002215698

In [44]:
# Probability that a respondent is a Democrat given that they are liberal
conditional(democrat, given=liberal)


0.5206403320240125

In [45]:
# Probability that Linda is a banker, given that she is female
conditional(banker, given=female)


0.02116102749801969

In [46]:
# Create a Boolean series for liberal Democrats
liberal_democrat = liberal & democrat

# Probability that Linda is a banker and a liberal Democrat, given that she is female
conditional(banker & liberal_democrat, given=female)


0.0023009316887329786

In [47]:
# Define young as respondents under 30
young = (gss['age'] < 30)

# Probability that a respondent is young
prob(young)

# Probability that a randomly chosen respondent is a young liberal
prob(young & liberal)


0.06579427875836884

In [48]:
# Probability that a young person is liberal
conditional(liberal, given=young)


0.338517745302714

In [49]:
# Define old as respondents 65 or older
old = (gss['age'] >= 65)

# Probability that a respondent is old
prob(old)

# Define conservative as respondents with political views Conservative, Slightly Conservative, or Extremely Conservative
conservative = (gss['polviews'] >= 5)

# Probability that a randomly chosen respondent is an old conservative
prob(old & conservative)


0.06701156421180766

In [50]:
# Probability that a conservative respondent is old
conditional(old, given=conservative)


0.19597721609113564